In [1]:
import numpy as np
import pandas as pd
import keras
import imageio
import os
import sagemaker
import matplotlib.pyplot as plt
from keras.applications import VGG16,ResNet50
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from PIL import Image
from sklearn.neighbors import NearestNeighbors
from glob import glob

Using TensorFlow backend.


In [2]:
import boto3,urllib.request
from sagemaker import get_execution_role

In [3]:
s3_bucket = 'narayani1'
prefix = 'reverseimagesearch'

In [4]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [5]:
rootdir = os.getcwd()

In [6]:
datadir = os.path.join(os.getcwd(),'ut-zap50k-images-square')
datadir

'/home/ec2-user/SageMaker/ut-zap50k-images-square'

In [7]:
%%time
pic_paths = []
pic_paths = [y for x in os.walk(datadir) for y in glob(os.path.join(x[0], '*.jpg'))]

CPU times: user 375 ms, sys: 174 ms, total: 549 ms
Wall time: 2.04 s


In [8]:
model = VGG16(weights='imagenet',include_top=True)

W0813 16:37:11.892215 140188081870656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 16:37:11.917585 140188081870656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 16:37:11.920557 140188081870656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 16:37:11.952161 140188081870656 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:39

In [9]:
def make_preds(smaller_set):
    images = [imageio.imread(i) for i in smaller_set]
    ims = np.array([np.array(Image.fromarray(i).resize((224,224))) for i in images])
    out = model.predict(ims) # (500,1000) np array
    return out

In [19]:
%%time
total = len(pic_paths) #s3
s = 500
df = pd.DataFrame()
for i in range(0,total//s+1):
    start = i*s
    fin = start + s
    if fin > total:
        fin = total
    smaller_set = [pic_paths[j] for j in range(start,fin)]
    #call predict
    if i < 5:
        out = make_preds(smaller_set)
        df = df.append(pd.DataFrame(out))

CPU times: user 19min 14s, sys: 1.61 s, total: 19min 15s
Wall time: 9min 48s


In [21]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,3.220697e-08,2.502751e-09,2.844303e-09,1.881100e-08,5.179037e-08,1.577017e-06,6.437785e-08,2.946148e-10,7.299469e-10,1.217509e-10,...,9.076304e-09,3.834441e-10,6.874908e-10,1.405343e-10,5.700783e-10,1.494771e-09,7.525984e-09,2.905650e-10,5.604487e-09,4.010277e-09
1,1.302339e-10,6.792949e-12,2.747681e-11,2.127728e-10,6.190710e-10,1.239769e-08,1.786325e-10,2.964413e-11,8.101334e-12,3.588683e-12,...,2.958230e-10,5.204434e-11,3.162977e-11,1.254328e-12,1.482424e-10,6.223489e-11,6.425988e-09,1.408956e-11,4.245674e-09,2.050493e-07
2,1.615343e-12,4.141338e-14,3.280376e-14,5.823341e-13,2.300089e-13,7.145049e-10,7.127837e-12,2.804335e-14,2.236784e-13,9.203087e-16,...,3.696794e-12,7.967438e-15,2.975961e-14,6.757184e-15,9.723022e-14,5.682956e-13,2.725190e-12,3.868362e-14,1.406703e-12,2.517933e-12
3,4.982113e-09,1.770395e-10,8.950612e-11,2.123098e-09,2.098042e-10,3.178967e-07,4.105873e-09,4.191087e-10,4.551753e-09,3.688193e-11,...,8.658361e-09,5.523259e-11,2.236338e-10,2.219870e-10,5.428916e-10,1.035841e-09,3.742644e-08,3.588555e-10,9.360546e-09,7.689429e-09
4,5.817769e-13,2.678934e-14,1.389731e-13,1.029916e-12,6.243282e-13,3.587281e-09,4.271903e-10,4.284249e-14,3.562169e-13,9.383928e-15,...,5.826411e-12,2.067677e-14,3.140768e-13,1.273405e-13,1.464783e-13,2.409784e-12,5.642385e-11,9.518415e-13,1.401631e-12,4.042779e-12


In [23]:
df.to_csv('points.csv',header=None,index=None)
sagemaker_session.upload_data(
                            path=os.path.join(rootdir,'points.csv'),
                            bucket=s3_bucket,
                            key_prefix='{}/{}'.format(prefix,'points'))

's3://narayani1/reverseimagesearch/points/points.csv'